In [8]:
#!/usr/bin/env python3
"""
ALTERNATIVE COLAB 1: Full Fine-tuning - FIXED VERSION
Clean code with no indentation errors
"""

print("="*80)
print("🚀 ALTERNATIVE COLAB 1: FULL FINE-TUNING - CODING")
print("="*80)

# INSTALL
print("\n📦 Installing...")
import subprocess
subprocess.run("pip install -q transformers datasets accelerate trl peft bitsandbytes", shell=True)
print("✅ Done!")

# IMPORTS
print("\n📚 Importing...")
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments
from datasets import load_dataset, Dataset
from trl import SFTTrainer
from peft import LoraConfig, get_peft_model
import torch

print("✅ Imported!")
print(f"GPU: {torch.cuda.is_available()}")

# LOAD MODEL
print("\n📥 Loading model...")
model = AutoModelForCausalLM.from_pretrained(
    "HuggingFaceTB/SmolLM-135M",
    device_map="auto",
    torch_dtype=torch.float16,
)
tokenizer = AutoTokenizer.from_pretrained("HuggingFaceTB/SmolLM-135M")
tokenizer.pad_token = tokenizer.eos_token
print("✅ Model loaded!")

# HIGH-RANK LORA
print("\n🔧 Adding high-rank LoRA...")
lora_config = LoraConfig(
    r=128,
    lora_alpha=256,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_dropout=0.1,
    bias="all",
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()
print("✅ LoRA added!")

# LOAD DATASET
print("\n📚 Loading Code Alpaca...")
dataset = load_dataset("sahil2801/CodeAlpaca-20k", split="train[:800]")
print(f"✅ Loaded {len(dataset)} examples!")

# FORMAT - CLEAN VERSION
print("\n🔧 Formatting...")

def format_coding(examples):
    formatted_texts = []
    for idx in range(len(examples['instruction'])):
        inst = examples['instruction'][idx]
        inp = examples['input'][idx] if examples['input'][idx] else ""
        out = examples['output'][idx]

        if inp.strip():
            txt = f"### Instruction:\n{inst}\n\n### Input:\n{inp}\n\n### Response:\n{out}"
        else:
            txt = f"### Instruction:\n{inst}\n\n### Response:\n{out}"

        formatted_texts.append(txt + tokenizer.eos_token)

    return {"text": formatted_texts}

formatted = format_coding(dataset)
train_dataset = Dataset.from_dict({"text": formatted["text"]})
print(f"✅ Formatted {len(train_dataset)} examples!")

# TRAIN
print("\n🚀 Training...")
training_args = TrainingArguments(
    output_dir="./full_ft_coding",
    num_train_epochs=1,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=8,
    learning_rate=3e-4,
    warmup_steps=20,
    logging_steps=10,
    save_strategy="no",
    fp16=True,
    report_to="none",
    max_steps=80,
)

trainer = SFTTrainer(model=model, args=training_args, train_dataset=train_dataset)
trainer.train()
print("\n✅ Training complete!")

# TEST
print("\n🧪 Testing...")
model.eval()

test_prompts = [
    "### Instruction:\nWrite a Python function to reverse a string\n\n### Response:\n",
    "### Instruction:\nCreate a function to calculate factorial\n\n### Response:\n",
]

for prompt in test_prompts:
    print(f"\n{prompt.split('Response:')[0].strip()}")
    print("Code:", end=" ")
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(**inputs, max_new_tokens=100, temperature=0.7, pad_token_id=tokenizer.eos_token_id)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True).split("Response:")[-1].strip()
    print(response)

# SAVE
print("\n💾 Saving...")
model.save_pretrained("./coding_full_ft_model")
tokenizer.save_pretrained("./coding_full_ft_model")

print("\n" + "="*80)
print("🎉 ALTERNATIVE COLAB 1 COMPLETE!")
print("="*80)
print("Summary:")
print("  ✓ Model: SmolLM-135M")
print("  ✓ Dataset: Code Alpaca (800 examples)")
print("  ✓ Method: High-rank LoRA (r=128)")
print("="*80)

🚀 ALTERNATIVE COLAB 1: FULL FINE-TUNING - CODING

📦 Installing...
✅ Done!

📚 Importing...
✅ Imported!
GPU: True

📥 Loading model...
✅ Model loaded!

🔧 Adding high-rank LoRA...
trainable params: 39,075,840 || all params: 173,590,848 || trainable%: 22.5103
✅ LoRA added!

📚 Loading Code Alpaca...
✅ Loaded 800 examples!

🔧 Formatting...
✅ Formatted 800 examples!

🚀 Training...


Adding EOS to train dataset:   0%|          | 0/800 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/800 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/800 [00:00<?, ? examples/s]

The model is already on multiple devices. Skipping the move to device specified in `args`.


Step,Training Loss
10,1.251100
20,0.986500
30,0.932900
40,0.924600
50,0.869600
60,0.789800
70,0.779000
80,0.781700


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



✅ Training complete!

🧪 Testing...

### Instruction:
Write a Python function to reverse a string

###
Code: def reverse_string(str):
    return str[::-1]

### Instruction:
Create a function to calculate factorial

###
Code: def factorial(n):
    if n == 0:
        return 1
    else:
        return n * factorial(n-1)

💾 Saving...

🎉 ALTERNATIVE COLAB 1 COMPLETE!
Summary:
  ✓ Model: SmolLM-135M
  ✓ Dataset: Code Alpaca (800 examples)
  ✓ Method: High-rank LoRA (r=128)
